In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from numpy import loadtxt
import pandas as pd
import numpy as np
from matplotlib import pyplot
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [2]:
df1 = pd.read_csv("../data/one-hotted-tedsd/updated_2016_3_v2.csv")
print("Finished processing ... df1");
df2 = pd.read_csv("../data/one-hotted-tedsd/updated_2016_2_v2.csv")
print("Finished processing ... df2");
df3 = pd.read_csv("../data/one-hotted-tedsd/update_2016_1_v3.csv")
print("Finished processing ... df3");
frames = [df1, df2, df3]
print("Finished processing ... frames");
original_df = pd.concat(frames)
print("This is the very original dataset size: " + str(original_df.shape))

Finished processing ... df1
Finished processing ... df2
Finished processing ... df3
Finished processing ... frames


/Users/jonahwu/miniconda3/envs/pysal2_1_release/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


This is the very original dataset size: (1458045, 248)


In [3]:
df_copy = original_df.copy(deep=True).sample(frac=1)
dataset = df_copy.to_numpy()

In [4]:
#Extract Features
dataFields = {}
for x in range(len(df_copy.columns)):
	dataFields.update({df_copy.columns[x]: x})
print(dataFields)
indx = dataFields["reason"]
Y = dataset[:,[indx]]
Y = np.select([(Y == 1)], [1] , default = 0) #Set to binary classification

featureWeights = pd.read_csv("../data/rankedFeatures.csv")
fields = list(featureWeights["field"])
desiredChars = [fields[x] for x in range(143)]
print(desiredChars)
indxsToDelete = []
for key, value in dataFields.items():
    if key not in desiredChars:
        indxsToDelete.append(value)

X = np.delete(dataset, indxsToDelete, 1)
print("Shape of dataset following feature deletion: ", X.shape)
print("Label Dimensions:", Y.shape)


{'ALCFLG': 0, 'AMPHFLG': 1, 'ARRESTS': 2, 'BARBFLG': 3, 'BENZFLG': 4, 'COKEFLG': 5, 'DSMCRIT': 6, 'HALLFLG': 7, 'HERFLG': 8, 'IDU': 9, 'INHFLG': 10, 'MARFLG': 11, 'METHFLG': 12, 'MTHAMFLG': 13, 'NOPRIOR': 14, 'OPSYNFLG': 15, 'OTCFLG': 16, 'OTHERFLG': 17, 'PCPFLG': 18, 'PSYPROB': 19, 'SEDHPFLG': 20, 'STIMFLG': 21, 'TRNQFLG': 22, 'age_approx': 23, 'alcdrug_alc': 24, 'alcdrug_both': 25, 'alcdrug_drug': 26, 'alcdrug_none': 27, 'daywait_approx': 28, 'division_enc': 29, 'division_esc': 30, 'division_ma': 31, 'division_mt': 32, 'division_ne': 33, 'division_pc': 34, 'division_sa': 35, 'division_territory': 36, 'division_wnc': 37, 'division_wsc': 38, 'educ_approx': 39, 'employ_full': 40, 'employ_nlf': 41, 'employ_part': 42, 'employ_unemployed': 43, 'employ_unknown': 44, 'ethnic_cuban': 45, 'ethnic_latino': 46, 'ethnic_mexican': 47, 'ethnic_none': 48, 'ethnic_puerto': 49, 'ethnic_unknown': 50, 'firstuse1_approx': 51, 'firstuse2_approx': 52, 'freq1_daily': 53, 'freq1_none': 54, 'freq1_some': 55, 

In [5]:

test_size = .05
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)


In [6]:
eval_set=[(X_train, y_train.ravel()), (X_test, y_test.ravel())]

In [7]:
model = XGBClassifier(learning_rate = 0.2, max_depth=5, n_estimators=500, verbosity=2)
model.fit(X_train, y_train.ravel(), early_stopping_rounds=25, eval_metric=["error", "logloss"],
    eval_set=eval_set, verbose=True)
print(model)

[18:27:36] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=5
[0]	validation_0-error:0.35970	validation_0-logloss:0.67244	validation_1-error:0.35819	validation_1-logloss:0.67247
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 25 rounds.
[18:27:43] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=5
[1]	validation_0-error:0.35203	validation_0-logloss:0.66007	validation_1-error:0.35159	validation_1-logloss:0.65875
[18:27:49] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[2]	validation_0-error:0.35203	validation_0-logloss:0.64764	validation_1-error:0.35158	validation_1-logloss:0.64785
[18:27:55] INFO: /Users/travis/build/dmlc/xgboost/src/tree/up

[31]	validation_0-error:0.32415	validation_0-logloss:0.59775	validation_1-error:0.32306	validation_1-logloss:0.59742
[18:33:56] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[32]	validation_0-error:0.32397	validation_0-logloss:0.59754	validation_1-error:0.32294	validation_1-logloss:0.59725
[18:34:07] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=5
[33]	validation_0-error:0.32382	validation_0-logloss:0.59729	validation_1-error:0.32294	validation_1-logloss:0.59695
[18:34:19] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[34]	validation_0-error:0.32316	validation_0-logloss:0.59691	validation_1-error:0.32202	validation_1-logloss:0.59652
[18:34:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra

[63]	validation_0-error:0.31599	validation_0-logloss:0.58993	validation_1-error:0.31569	validation_1-logloss:0.59010
[18:40:58] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[64]	validation_0-error:0.31574	validation_0-logloss:0.58976	validation_1-error:0.31556	validation_1-logloss:0.58995
[18:41:16] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[65]	validation_0-error:0.31570	validation_0-logloss:0.58958	validation_1-error:0.31567	validation_1-logloss:0.58979
[18:41:35] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[66]	validation_0-error:0.31560	validation_0-logloss:0.58946	validation_1-error:0.31571	validation_1-logloss:0.58967
[18:41:47] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra

[95]	validation_0-error:0.31370	validation_0-logloss:0.58633	validation_1-error:0.31453	validation_1-logloss:0.58683
[18:53:59] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[96]	validation_0-error:0.31371	validation_0-logloss:0.58628	validation_1-error:0.31455	validation_1-logloss:0.58680
[18:54:27] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=5
[97]	validation_0-error:0.31355	validation_0-logloss:0.58613	validation_1-error:0.31428	validation_1-logloss:0.58665
[18:54:54] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 60 extra nodes, 0 pruned nodes, max_depth=5
[98]	validation_0-error:0.31353	validation_0-logloss:0.58604	validation_1-error:0.31423	validation_1-logloss:0.58658
[18:55:22] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra

[127]	validation_0-error:0.31203	validation_0-logloss:0.58410	validation_1-error:0.31314	validation_1-logloss:0.58503
[19:09:29] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[128]	validation_0-error:0.31202	validation_0-logloss:0.58406	validation_1-error:0.31299	validation_1-logloss:0.58502
[19:09:59] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[129]	validation_0-error:0.31197	validation_0-logloss:0.58403	validation_1-error:0.31296	validation_1-logloss:0.58500
[19:10:26] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 58 extra nodes, 0 pruned nodes, max_depth=5
[130]	validation_0-error:0.31195	validation_0-logloss:0.58401	validation_1-error:0.31289	validation_1-logloss:0.58499
[19:10:53] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 e

...

[351]	validation_0-error:0.30629	validation_0-logloss:0.57609	validation_1-error:0.30875	validation_1-logloss:0.57986
[20:09:05] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[352]	validation_0-error:0.30629	validation_0-logloss:0.57606	validation_1-error:0.30867	validation_1-logloss:0.57985
[20:09:18] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[353]	validation_0-error:0.30628	validation_0-logloss:0.57603	validation_1-error:0.30866	validation_1-logloss:0.57984
[20:09:30] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[354]	validation_0-error:0.30630	validation_0-logloss:0.57600	validation_1-error:0.30852	validation_1-logloss:0.57984
[20:09:42] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 e

[383]	validation_0-error:0.30587	validation_0-logloss:0.57542	validation_1-error:0.30867	validation_1-logloss:0.57950
[20:15:35] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[384]	validation_0-error:0.30585	validation_0-logloss:0.57539	validation_1-error:0.30859	validation_1-logloss:0.57949
[20:15:48] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 50 extra nodes, 0 pruned nodes, max_depth=5
[385]	validation_0-error:0.30585	validation_0-logloss:0.57538	validation_1-error:0.30859	validation_1-logloss:0.57948
[20:16:00] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 56 extra nodes, 0 pruned nodes, max_depth=5
[386]	validation_0-error:0.30580	validation_0-logloss:0.57534	validation_1-error:0.30853	validation_1-logloss:0.57945
[20:16:12] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 e

[415]	validation_0-error:0.30538	validation_0-logloss:0.57473	validation_1-error:0.30831	validation_1-logloss:0.57919
[20:22:12] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 54 extra nodes, 0 pruned nodes, max_depth=5
[416]	validation_0-error:0.30538	validation_0-logloss:0.57472	validation_1-error:0.30848	validation_1-logloss:0.57919
[20:22:23] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[417]	validation_0-error:0.30535	validation_0-logloss:0.57470	validation_1-error:0.30841	validation_1-logloss:0.57918
[20:22:33] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 56 extra nodes, 0 pruned nodes, max_depth=5
[418]	validation_0-error:0.30535	validation_0-logloss:0.57468	validation_1-error:0.30838	validation_1-logloss:0.57918
[20:22:46] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 56 e

[447]	validation_0-error:0.30488	validation_0-logloss:0.57410	validation_1-error:0.30771	validation_1-logloss:0.57880
[20:28:47] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[448]	validation_0-error:0.30489	validation_0-logloss:0.57408	validation_1-error:0.30772	validation_1-logloss:0.57880
[20:28:59] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 44 extra nodes, 0 pruned nodes, max_depth=5
[449]	validation_0-error:0.30487	validation_0-logloss:0.57407	validation_1-error:0.30770	validation_1-logloss:0.57878
[20:29:15] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[450]	validation_0-error:0.30482	validation_0-logloss:0.57403	validation_1-error:0.30759	validation_1-logloss:0.57877
[20:29:34] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 60 e

[479]	validation_0-error:0.30447	validation_0-logloss:0.57350	validation_1-error:0.30753	validation_1-logloss:0.57860
[20:37:27] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[480]	validation_0-error:0.30447	validation_0-logloss:0.57348	validation_1-error:0.30753	validation_1-logloss:0.57858
[20:37:43] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[481]	validation_0-error:0.30444	validation_0-logloss:0.57346	validation_1-error:0.30745	validation_1-logloss:0.57856
[20:37:57] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 extra nodes, 0 pruned nodes, max_depth=5
[482]	validation_0-error:0.30441	validation_0-logloss:0.57343	validation_1-error:0.30731	validation_1-logloss:0.57854
[20:38:18] INFO: /Users/travis/build/dmlc/xgboost/src/tree/updater_prune.cc:98: tree pruning end, 62 e

In [12]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# precision tp / (tp + fp)
precision = precision_score(y_test, predictions)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, predictions)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, predictions)
print('F1 score: %f' % f1)


print("Now for training set")
y_pred = model.predict(X_train)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_train, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# precision tp / (tp + fp)
precision = precision_score(y_train, predictions)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_train, predictions)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_train, predictions)
print('F1 score: %f' % f1)


Accuracy: 69.27%
Precision: 0.700167
Recall: 0.435906
F1 score: 0.537302
Now for training set
Accuracy: 69.59%
Precision: 0.707850
Recall: 0.443386
F1 score: 0.545241
